# importing is weird in notebooks

In [25]:
#code from utils.py
import pandas as pd

def clean_data(df):
    """ This function drops duplicates once more to be sure
        drops the column property_ID since it does not affect price and is an immovlan code
        converts our datatypes (to the right type)
        drops properties without prices
        """
    
    # Drop duplicates (safety)
    df = df.drop_duplicates()

    # Drop useless columns
    df = df.drop(columns=["property_ID"], errors="ignore")
    df = df.drop(columns=["locality_name"], errors="ignore")

    # Convert datatypes
    df = df.apply(lambda x: x.astype("Int64") if x.dtype == float and (x.dropna() % 1 == 0).all() else x)

    # Explicit type conversions
    dtype_map = {
        "locality_name": object,
        "province": object,
        "region": object,
        "type": "category",
        "subtype": "category",
        "state_of_building": "category",
        "postal_code": "category"
        
    }
    for col, dtype in dtype_map.items():
        if col in df.columns:
            df[col] = df[col].astype(dtype)

    # Drop rows without price
    df = df.dropna(subset=["price (€)"])

    #convert nullable dtypes pd.NA (pandas nans) to np.nan (numpy nans)
    #this must be done because later on transformers can not handle pd.NA
    df = df.replace({pd.NA: np.nan})

    return df

# Step 1 : import and load data

In [26]:
import pandas as pd
import numpy as np

#load dataframe from csv
df = pd.read_csv("../data/filtered_final_cleaned_data.csv")

# clean the dataset
df_clean = clean_data(df)

#check if conversion went well and how many rows now
print(df_clean.dtypes)
print(df_clean.shape)



postal_code                      category
type                             category
subtype                          category
price (€)                           int64
number_of_bedrooms                  Int64
living_area (m²)                    int64
equiped_kitchen (yes:1, no:0)       int64
furnished (yes:1, no:0)             int64
open_fire (yes:1, no:0)             int64
terrace (yes:1, no:0)               int64
terrace_area (m²)                   Int64
garden (yes:1, no:0)                int64
number_facades                      Int64
swimming_pool (yes:1, no:0)         int64
state_of_building                category
province                           object
region                             object
dtype: object
(14545, 17)


In [27]:
#test if pd.NA are still there
for col in df_clean.columns:
    if df_clean[col].isna().any():
        if df_clean[col].apply(lambda x: isinstance(x, pd._libs.missing.NAType)).any():
                print("pd.NA still present in column:", col)

recheck the missing values

In [28]:
import pandas as pd

# Check for missing values
missing_data = df_clean.isnull().sum()

# Put the missing values per column in percentage
missing_summary = pd.DataFrame({
    'Missing Values': df_clean.isnull().sum(),
    'Percentage (%)': (df_clean.isnull().sum() / len(df)) * 100
})

# Round off the percentages
missing_summary['Percentage (%)'] = missing_summary['Percentage (%)'].round(0)
display(missing_summary)

,Missing Values,Percentage (%)
postal_code,0,0.0
type,0,0.0
subtype,0,0.0
price (€),0,0.0
number_of_bedrooms,148,1.0
living_area (m²),0,0.0
"equiped_kitchen (yes:1, no:0)",0,0.0
"furnished (yes:1, no:0)",0,0.0
"open_fire (yes:1, no:0)",0,0.0
"terrace (yes:1, no:0)",0,0.0


# STEP 2: identify feature_types

In [29]:
numeric_features = df_clean.select_dtypes(include ="number").columns.tolist()
numeric_features.remove("price (€)") #target is not a feature!
categorical_features = df_clean.select_dtypes(include=["object","category","string"]).columns.tolist()

#check
print(numeric_features)
print(categorical_features)

['number_of_bedrooms', 'living_area (m²)', 'equiped_kitchen (yes:1, no:0)', 'furnished (yes:1, no:0)', 'open_fire (yes:1, no:0)', 'terrace (yes:1, no:0)', 'terrace_area (m²)', 'garden (yes:1, no:0)', 'number_facades', 'swimming_pool (yes:1, no:0)']
['postal_code', 'type', 'subtype', 'state_of_building', 'province', 'region']


# STEP 3: Train_Test Splitting

In [30]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd


#split into features and target
X = df_clean.drop(columns=["price (€)"])
y = df_clean["price (€)"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#some checks
print("Train shape:", X_train.shape)
print("Test Shape:", X_test.shape)

#recheck dtypes
print(X_train.dtypes)

#retest if pd.NA are still in X_train again
for col in X_train.columns:
    if X_train[col].isna().any():
        if X_train[col].apply(lambda x: isinstance(x, pd._libs.missing.NAType)).any():
                print("pd.NA still present in column:", col)



Train shape: (11636, 16)
Test Shape: (2909, 16)
postal_code                      category
type                             category
subtype                          category
number_of_bedrooms                  Int64
living_area (m²)                    int64
equiped_kitchen (yes:1, no:0)       int64
furnished (yes:1, no:0)             int64
open_fire (yes:1, no:0)             int64
terrace (yes:1, no:0)               int64
terrace_area (m²)                   Int64
garden (yes:1, no:0)                int64
number_facades                      Int64
swimming_pool (yes:1, no:0)         int64
state_of_building                category
province                           object
region                             object
dtype: object


# STEP 4: Build preproc and full pipeline

In [31]:
#before we decide on what encoder and so on we need to know how many categories each categorical feature has
for col in categorical_features:
    print(col, df_clean[col].nunique())

postal_code 903
type 2
subtype 15
state_of_building 9
province 11
region 3


In [32]:
#move this code to pipeline.py
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

def build_minimal_pipeline (X_train, y_train):
    """takes a cleaned dataframe and returns a trained minimal pipeline (BASELINE)"""
    pipeline = Pipeline([("model", LinearRegression())])
    pipeline.fit(X_train, y_train)
    return pipeline

def build_preprocessing_pipeline(numeric_features, categorical_features):
    """build a first full preprocessing pipeline and improve later per feature if necessary
        numeric: impute median + scale
        categorical: impute most_frequent + OHE"""
    
    #numeric preprocessing
    numeric_transformer = Pipeline(steps=[("imputer", SimpleImputer (strategy = "median")), ("scaler", StandardScaler())])

    #categorical preprocessing
    categorical_transformer = Pipeline(steps=[("imputer", SimpleImputer(strategy="most_frequent")), ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))])

    #combine preproc for num & cat features
    preprocessor = ColumnTransformer(transformers=[("num", numeric_transformer, numeric_features),("cat", categorical_transformer, categorical_features)])

    return preprocessor

def build_full_pipeline(preprocessor):
    """attach a regression model to the preproccesing pipeline
        for now : Linear Regression (baseline)
        later: RF, XGBOOST, SVM"""
    
    full_pipeline = Pipeline(steps=[("preprocessing", preprocessor), ("model", LinearRegression())])

    return full_pipeline


In [33]:
#preproc and full pipeline now
preprocessor = build_preprocessing_pipeline(numeric_features, categorical_features)
pipeline_full = build_full_pipeline(preprocessor)

pipeline_full.fit(X_train, y_train)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


# STEP 5: EVALUATE

In [34]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

y_train_pred = pipeline_full.predict(X_train)
y_test_pred = pipeline_full.predict(X_test)

R2_train = r2_score(y_train, y_train_pred)
MAE_train = mean_absolute_error(y_train, y_train_pred)
MSE_train = mean_squared_error(y_train, y_train_pred)

R2_test = r2_score(y_test, y_test_pred)
MAE_test = mean_absolute_error(y_test, y_test_pred)
MSE_test = mean_squared_error(y_test, y_test_pred)


print("Training Metrics:")
print("R² train:", R2_train)
print("MAE train:", MAE_train)
print("MSE train:", MSE_train)

print("Testing Metrics")
print("R² test:", R2_test)
print("MAE test:", MAE_test)
print("MSE test:", MSE_test)
print("")

if R2_train >= R2_test:
    print("model is overfitting")
elif R2_train and R2_test < 0.5:
    print("model is weak and underfitting")
else:
    print("model is a good fit")





Training Metrics:
R² train: 0.6547306235973126
MAE train: 79613.72042421816
MSE train: 25543703173.364784
Testing Metrics
R² test: 0.5620619859878329
MAE test: 87160.80069643211
MSE test: 26229978653.459534

model is overfitting
